# Visit PostISRCCD Images from Selected visits LSST in Auxtel

- author Sylvie Dagoret-Campagne
- creation date 2024-05-06
- last update 2024-05-15
- affiliation : IJCLab
- kernel : **w_2024_16**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS


import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'x-large'
plt.rcParams['axes.titlesize'] = 'x-large'
plt.rcParams['xtick.labelsize']= 'x-large'
plt.rcParams['ytick.labelsize']= 'x-large'

In [ ]:
import gc

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils

In [ ]:
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import lsst.afw.table as afwTable
import lsst.geom as geom

In [ ]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
transform = AsinhStretch() + PercentileInterval(99.)

### Import rubinsimphot

In [ ]:
machine_name = os.uname().nodename
path_rubinsimphot = "repos/repos_w_2024_17/rubinsimphot/src"
if 'sdf' in machine_name:
    #machine_name_usdf = 'sdfrome001'
    print("Set environment for USDF")
    newpythonpath = os.path.join(os.getenv("HOME"),path_rubinsimphot)
    sys.path.append(newpythonpath)
elif 'dagoret-nb' in machine_name:
    print("Set environment for USDF Rubin Science Platform")
    newpythonpath = os.path.join(os.getenv("HOME"),path_rubinsimphot)
    sys.path.append(newpythonpath)    
elif 'mac' in machine_name:
    print("Be sure to run this notebook in conda environment named conda_py310")
else:
    print(f"Your current machine name is {machine_name}. Check your python environment")

In [ ]:
from rubinsimphot.phot_utils import Bandpass, Sed
from rubinsimphot.data import get_data_dir

In [ ]:
fdir = get_data_dir()
path_rubin_sim_throughput = os.path.join(fdir, 'throughputs', 'auxtel')

In [ ]:
import sys
sys.path.append('../lib')
# This package encapsulate the calculation on calibration used in this nb
from libPhotometricCorrections_auxtel import *

In [ ]:
from importlib.metadata import version
the_ver = version('getObsAtmo')
print(f"Version of getObsAtmo : {the_ver}")

In [ ]:
from getObsAtmo import ObsAtmo
emul_atm = ObsAtmo()

In [ ]:
WL = emul_atm.GetWL()

In [ ]:
am0 =1.20    # airmass
pwv0 = 5.0  # Precipitable water vapor vertical column depth in mm
oz0 = 300.  # Ozone vertical column depth in Dobson Unit (DU)
ncomp=1     # Number of aerosol components
tau0= 0.04 # Vertical Aerosol depth (VAOD) 
beta0 = 1.2 # Aerosol Angstrom exponent
pc = PhotometricCorrections(am0,pwv0,oz0,tau0,beta0)

In [ ]:
all_Sigb = {}

for index,f in enumerate(filter_tagnames): 
    wl = pc.bandpass_inst[f].wavelen
    tr = pc.bandpass_inst[f].sb
    Sigb = np.trapz(tr/wl,wl)
    all_Sigb[f] = Sigb 

In [ ]:
pd.DataFrame(all_Sigb, index=['Sigmab']).round(4)

## Import statements that we will need later

Let's make a new plot and metric tool, we'll base it on the example in the getting started guide.

In [ ]:
# INSERT YOUR collection and tract
# for rehearsal use collection 2 which have CCDvisit
butlerRoot = "/repo/embargo"

collection1 = 'LSSTComCamSim/runs/nightlyvalidation/20240402/d_2024_03_29/DM-43612'
collection2 = 'LSSTComCamSim/runs/nightlyvalidation/20240403/d_2024_03_29/DM-43612'
collection3 = 'LSSTComCamSim/runs/nightlyvalidation/20240404/d_2024_03_29/DM-43612'
#collection = 'LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240311/w_2024_10/PREOPS-4985'
collection = 'LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240414/w_2024_15/PREOPS-5069' # COMPLETED
collectionn = collection
#collections = [collection1,collection2,collection3]
collections = [collection]
collectionStr = collectionn.replace("/", "_")
fn_ccdVisit_tracts_patches = f"ccdVisittractpatch_{collectionStr}.csv"
instrument = 'LATISS'
skymapName = "latiss_v1"
where_clause = "instrument = \'" + instrument+ "\'"
tract = 3864 # mostly for light-curves
patch_sel = 236
band = 'g'
#tract = 5615
# tract = 5634 # interesting to view calib parameters
suptitle = collectionStr + f" inst = {instrument} tract = {tract}"

In [ ]:
#dataId = {"skymap": "latiss_v1", "tract": 5615, "instrument": "LATISS"}
dataId = {"skymap": skymapName, "tract": tract, "instrument": instrument}
repo = '/sdf/group/rubin/repo/oga/'
butler = dafButler.Butler(repo)
#t = Butler.get(table_sel, dataId=dataId, collections=collections)
registry = butler.registry

In [ ]:
skymap = butler.get('skyMap', skymap=skymapName, collections=collections)

In [ ]:
def remove_figure(fig):
    """
    Remove a figure to reduce memory footprint.

    Parameters
    ----------
    fig: matplotlib.figure.Figure
        Figure to be removed.

    Returns
    -------
    None
    """
    # get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()       # clear the figure
    plt.close(fig)  # close the figure
    gc.collect()    # call the garbage collector

In [ ]:
def get_corners_radec(wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    corners_ra, corners_dec in decimal degrees
    """

    corners_x = [bbox.beginX, bbox.beginX, bbox.endX, bbox.endX]
    corners_y = [bbox.beginY, bbox.endY, bbox.endY, bbox.beginY]
    corners_ra = []
    corners_dec = []
    for i in range(4):
        radec = wcs.pixelToSky(corners_x[i], corners_y[i])
        corners_ra.append(radec.getRa().asDegrees())
        corners_dec.append(radec.getDec().asDegrees())
    
    return corners_ra, corners_dec

In [ ]:
def convert_fluxtomag(x) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    return -2.50 * np.log10(x) + 31.4

## Get pixel scale

In [ ]:
import lsst.geom as geom
import lsst.sphgeom

skymap = butler.get('skyMap', skymap=skymapName, collections=collections )
tractInfo = skymap.generateTract(tract)
for patch in tractInfo:    
    patchID = patch.getSequentialIndex()
        
    ibb=patch.getInnerBBox()
    tWCS=tractInfo.getWcs()
       
    # loop on the 4 corners
    for icorn,corner in enumerate(ibb.getCorners()):
        p = geom.Point2D(corner.getX(), corner.getY())
        coord = tWCS.pixelToSky(p)


In [ ]:
tWCS

In [ ]:
#arcsec/pixel
pixel_scale = tWCS.getPixelScale().asArcseconds()

## Selected visits

In [ ]:
if 1:
    # get the csv file produced by stat_on_visits_LSSTAuxtel.ipynb in ../Visits
    #file_selected_visits = "../Visits/ccdVisittractpatch_LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240311_w_2024_10_PREOPS-4985.csv"
    file_selected_visits = os.path.join("../Visits",fn_ccdVisit_tracts_patches)
    
    df_myselectedvisits = pd.read_csv(file_selected_visits,index_col=0)

    # select the band
    cut = (df_myselectedvisits.band == band) &  (df_myselectedvisits.tractID == tract) & (df_myselectedvisits.patchID == patch_sel)
    df_myselectedvisits = df_myselectedvisits[cut]

    #move the visitid as a column not to loose it during the merge 
    df_myselectedvisits.reset_index(inplace=True) 


In [ ]:
index = 0
visitId = df_myselectedvisits.iloc[index]['visitId']

Note here:
- zeroPoint is in ABMag relative to a flux of 1ADU/sec
- skyBg is in ADU 
- skyNoise is in ADU 

In [ ]:
df_myselectedvisits.iloc[index]

In [ ]:
datasetType = 'postISRCCD'
dataId = {'visit': visitId, 'instrument':instrument , 'detector': 0}
datasetRefs = registry.queryDatasets(datasetType, dataId=dataId, collections  = collections)

for i, ref in enumerate(datasetRefs):
    print(ref.dataId)
    print("band:", ref.dataId['band'])
    band = ref.dataId['band']
    img= butler.get(ref)
    md = img.getMetadata().toDict()

    list_of_keys = list(md.keys())
    for key in list_of_keys:
        if "CALIB" in key:
            print(key,"::",md[key])
        if "unit" in key or "UNIT" in key:
            print(key,"::",md[key])
        if "gain" in key or "GAIN" in key:
            print(key,"::",md[key])

In [ ]:
#md

## One postISRCCD and One Calexp

In [ ]:
def getvmivmaxfromimg(arr):
    data_flat = arr.flatten()
    med = np.median(data_flat)
    sigMad = 1.4826 * np.median(np.fabs(data_flat - med))
    vmin = med - 3 * sigMad
    vmax = med + 3 * sigMad
    return vmin,vmax

In [ ]:
%matplotlib widget

calexp = butler.get('calexp', **dataId,collections=collections)
bkgd  = butler.get('calexpBackground', **dataId,collections=collections)
# get the calibration constant  : expo_photocalibconstant_mean in nJ per ADU. To convert it in AB-Mag : convert_fluxtomag(expo_photocalibconstant_mean) --> zero point
calexp_info = calexp.getInfo()
photocalib = calexp_info.getPhotoCalib()
expo_photocalibconstant_mean = photocalib.getCalibrationMean()

mask = calexp.mask.array
the_mask = np.where(mask==0,1,0)
img1 = calexp.getImage()
vmin1,vmax1 = getvmivmaxfromimg(img1.array)
print(calexp.getPhotoCalib())

datasetRefs = registry.queryDatasets('postISRCCD', dataId=dataId, collections  = collections)
for i, ref in enumerate(datasetRefs):
    print(ref.dataId)
    expo  = butler.get(ref)
    img2 = expo.getImage()
#wcs = img.getWcs()
#bbox = img.getBBox()
#corners_ra, corners_dec = get_corners_radec(wcs, bbox)
#extent = [np.min(corners_ra),np.max(corners_ra),np.min(corners_dec),np.max(corners_dec)]
vmin2,vmax2 = getvmivmaxfromimg(img2.array)


fig,axs = plt.subplots(3,2,figsize=(16,18))

axs = axs.flatten()

ax1 = axs[0]

im1=ax1.imshow(img1.array, origin="lower",vmin=vmin1, vmax=vmax1, cmap="gray")
divider = make_axes_locatable(ax1)
cax1 = divider.append_axes("right", size="5%", pad=0.05)
the_title = f"Calexp : visit = {visitId}, band = {band}"
ax1.set_title(the_title)
ax1.grid()
ax1.set_aspect('equal')
plt.colorbar(im1, cax=cax1,ax=ax1)


ax2 = axs[1]
im2=ax2.imshow(img2.array, origin="lower",vmin=vmin2, vmax=vmax2, cmap="gray")
divider = make_axes_locatable(ax2)
cax2 = divider.append_axes("right", size="5%", pad=0.05)
the_title = f"postISRCCD : visit = {visitId}, band = {band}"
ax2.set_title(the_title)
ax2.grid()
ax2.set_aspect('equal')
plt.colorbar(im2, cax=cax2,ax=ax2)

ax3 = axs[2]
arr3 = img1.array*the_mask
vmin3,vmax3 = getvmivmaxfromimg(arr3)
im3=ax3.imshow(arr3, origin="lower",vmin=vmin3, vmax=vmax3, cmap="gray")
divider = make_axes_locatable(ax3)
cax3 = divider.append_axes("right", size="5%", pad=0.05)
the_title = f"mask Calexp : visit = {visitId}, band = {band}"
ax3.set_title(the_title)
ax3.grid()
ax3.set_aspect('equal')
plt.colorbar(im3, cax=cax3,ax=ax3)

ax4 = axs[3]
arr4 = img2.array*the_mask
vmin4,vmax4 = getvmivmaxfromimg(arr4)
im4=ax4.imshow(arr4, origin="lower",vmin=vmin4, vmax=vmax4, cmap="gray")
divider = make_axes_locatable(ax4)
cax4 = divider.append_axes("right", size="5%", pad=0.05)
the_title = f"mask postISRCCD : visit = {visitId}, band = {band}"
ax4.set_title(the_title)
ax4.grid()
ax4.set_aspect('equal')
plt.colorbar(im4, cax=cax4,ax=ax4)


ax5 = axs[4]
arr5 = (calexp.getImage().array+ bkgd.getImage().array)
vmin5,vmax5 = getvmivmaxfromimg(arr5)
im5=ax5.imshow(arr5, origin="lower",vmin=vmin5, vmax=vmax5, cmap="gray")
divider = make_axes_locatable(ax5)
cax5 = divider.append_axes("right", size="5%", pad=0.05)
the_title = f"calexp+bkg/calib : visit = {visitId}, band = {band}"
ax5.set_title(the_title)
ax5.grid()
ax5.set_aspect('equal')
plt.colorbar(im5, cax=cax5,ax=ax5)


ax6 = axs[5]
arr6 = (calexp.getImage().array+ bkgd.getImage().array) - img2.array
vmin6,vmax6 = getvmivmaxfromimg(arr6)
im6=ax6.imshow(arr6, origin="lower",vmin=vmin6, vmax=vmax6, cmap="gray")
divider = make_axes_locatable(ax6)
cax6 = divider.append_axes("right", size="5%", pad=0.05)
the_title = f"calexp+bkg-postisr : visit = {visitId}, band = {band}"
ax6.set_title(the_title)
ax6.grid()
ax6.set_aspect('equal')
plt.colorbar(im6, cax=cax6,ax=ax6)



plt.suptitle(collections)
plt.tight_layout()
plt.show()
#remove_figure(fig)

- Factor by which ADU x this factor = Flux in nJy

In [ ]:
photocalib 

- convert this factor in ABMag, related to zeropoint in CCDVisitTable

In [ ]:
print(expo_photocalibconstant_mean,convert_fluxtomag(expo_photocalibconstant_mean) )

### Calexp info

In [ ]:
calexp_info = calexp.getInfo()
calexp_info.getMetadata().toDict() 

## Many PostISRCCD

In [ ]:
df_myselectedvisits.tail(100)

In [ ]:
NROWS = 10
NCOLS = 3
NIMGS = NROWS*NCOLS
index0 = 160

In [ ]:
fig,axs = plt.subplots(NROWS,NCOLS,figsize=(NCOLS*4,NROWS*3))

for index,ax in enumerate(axs.flatten()):

    visitId = df_myselectedvisits.iloc[index0+index]['visitId']
    dataId = {'visit': visitId, 'instrument':instrument , 'detector': 0}
    datasetType = 'postISRCCD'

    datasetRefs = registry.queryDatasets(datasetType, dataId=dataId, collections  = collections)
    for i, ref in enumerate(datasetRefs):
        print(ref.dataId)
        expo  = butler.get(ref)
        img = expo.getImage()

    vmin,vmax = getvmivmaxfromimg(img.array)


    im=ax.imshow(img.array, origin="lower",vmin=vmin, vmax=vmax, cmap="gray")
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="3%", pad=0.05)
    the_title = f"{visitId},{band}"
    ax.set_title(the_title)
    ax.grid()
    ax.set_aspect('equal')
    plt.colorbar(im, cax=cax,ax=ax)

plt.suptitle(collections)
plt.tight_layout()
plt.show()
#remove_figure(fig)

## Background flux (ADU)

In [ ]:
fig,axs = plt.subplots(NROWS,NCOLS,figsize=(NCOLS*3,NROWS*3))

for index,ax in enumerate(axs.flatten()):
  

    visitId = df_myselectedvisits.iloc[index0+index]['visitId']
    dataId = {'visit': visitId, 'instrument':instrument , 'detector': 0}

    calexp = butler.get('calexp', **dataId,collections=collections)
    calexp_info = calexp.getInfo()
    photocalib = calexp_info.getPhotoCalib()
    expo_photocalibconstant_mean = photocalib.getCalibrationMean()
    del calexp
    
    datasetType = 'postISRCCD'
    datasetRefs = registry.queryDatasets(datasetType, dataId=dataId, collections  = collections)
    for i, ref in enumerate(datasetRefs):
        print(ref.dataId)
        expo  = butler.get(ref)
        img = expo.getImage()

    vmin,vmax = getvmivmaxfromimg(img.array)

    data_flat = img.array.flatten()
    mu = np.mean(data_flat)
    med = np.median(data_flat)
    std = np.std(data_flat)
    sigMad = 1.4826 * np.median(np.fabs(data_flat - med))
    vmin = med - 5 * sigMad
    vmax = med + 5 * sigMad

    
    textstr = '\n'.join((
    r'$\mu=%.2f$ ADU/pix' % (mu, ),
    r'$\mathrm{med}=%.2f$ ADU/pix' % (med, ),
    r'$\mathrm{std}=%.2f$ ADU/pix' % (std, ),
    r'$\mathrm{med}_{calib}=%.2f$ mag/arcsec2' % (convert_fluxtomag(med*expo_photocalibconstant_mean/pixel_scale**2), ),
    r'$\sigma=%.2f$ ADU/pix' % (sigMad, ),
    r'$\sigma_{calib}=%.1f$ mag/arcsec2' % (convert_fluxtomag(sigMad*expo_photocalibconstant_mean/pixel_scale**2), )))

    histdata = ax.hist(data_flat,bins=50,range=(vmin,vmax),histtype="step",color="b",lw=2)
    
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=10,verticalalignment='top', bbox=props)
    
    the_title = f"{visitId},{band}"
    ax.set_title(the_title)
    ax.set_xlabel("Background flux (ADU/pixel)")
    ax.grid()
    ax.set_ylim(0.,histdata[0].max()*2.)
  
plt.suptitle(collections)
plt.tight_layout()
plt.show()
#remove_figure(fig)

## Background flux (ADU) in not masked pixels

In [ ]:
fig,axs = plt.subplots(NROWS,NCOLS,figsize=(NCOLS*3,NROWS*3))

for index,ax in enumerate(axs.flatten()):
    visitId = df_myselectedvisits.iloc[index0+index]['visitId']
    dataId = {'visit': visitId, 'instrument':instrument , 'detector': 0}
    calexp = butler.get('calexp', **dataId,collections=collections)
    mask = calexp.mask.array
    the_mask = np.where(mask==0,1,0)

    calexp_info = calexp.getInfo()
    photocalib = calexp_info.getPhotoCalib()
    expo_photocalibconstant_mean = photocalib.getCalibrationMean()

    datasetType = 'postISRCCD'
    datasetRefs = registry.queryDatasets(datasetType, dataId=dataId, collections  = collections)
    for i, ref in enumerate(datasetRefs):
        print(ref.dataId)
        expo  = butler.get(ref)
        img = expo.getImage()

    
    img = bkgd.getImage().array* the_mask

    data_flat = img.flatten()
    data_flat_nozero = data_flat[~(data_flat==0)]
    mu = np.mean(data_flat_nozero )
    med = np.median(data_flat_nozero )
    sigMad_nozero  = 1.4826 * np.median(np.fabs(data_flat_nozero  - med))
    vmin = med - 5 * sigMad_nozero 
    vmax = med + 5 * sigMad_nozero 

    
    textstr = '\n'.join((
    r'$\mu=%.2f$ ADU/pix' % (mu, ),
    r'$\mathrm{med}=%.2f$ ADU/pix' % (med, ),
    r'$\mathrm{med}_{calib}=%.2f$ mag/arcsec2' % (convert_fluxtomag(med*expo_photocalibconstant_mean/pixel_scale**2), ),
    r'$\sigma=%.2f$ ADU/pix' % (sigMad_nozero , ),
    r'$\sigma_{calib}=%.1f$ mag/arcsec2' % (convert_fluxtomag(sigMad_nozero*expo_photocalibconstant_mean/pixel_scale**2 ), )))

    histdata = ax.hist(data_flat,bins=50,range=(vmin,vmax),histtype="step",color="b",lw=2)
    
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=10,verticalalignment='top', bbox=props)
    
    the_title = f"{visitId},{band}"
    ax.set_title(the_title)
    ax.set_xlabel("Background flux (ADU/pixel)")
    ax.grid()
    ax.set_ylim(0.,histdata[0].max()*2.)
  
plt.suptitle(collections)
plt.tight_layout()
plt.show()
#remove_figure(fig)